## Librerías

In [19]:
# Librerías de ciencia de datos 
import numpy as np
import pandas as pd
import sqlite3 as sql

# para análisis interactivo
from ipywidgets import interact, Output

# Librerías para preprocesamiento 
from sklearn.preprocessing import MinMaxScaler

# Librerías para análisis interactivo 
from ipywidgets import interact 
from sklearn import neighbors 
import joblib

# Paquete para sistemas de recomendación surprise 
from surprise import Reader, Dataset
from surprise.model_selection import cross_validate, GridSearchCV
from surprise import KNNBasic, KNNWithMeans, KNNWithZScore, KNNBaseline
from surprise.model_selection import train_test_split

## Carga de datos

In [2]:
#### Conectar #######
conn=sql.connect('data/db_movies')
cur=conn.cursor()

In [3]:
#ver tablas
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
cur.fetchall()

[('ratings',),
 ('movies',),
 ('usuarios_sel',),
 ('movies_sel',),
 ('ratings_final',),
 ('movie_final',),
 ('full_ratings',),
 ('reco',)]

## 3 Sistema de recomendación basado en contenido KNN Con base en todo lo visto por el usuario

### Top 10 de las películas más similares a las vistas por el usuario y con una que le haya dado una calificación mayor a 3.5.

In [4]:
# Vamos a trabajar con el dataframe escalado y con dummies df_dum 
df_dum = pd.read_csv('data/df_dum.csv')
movies = pd.read_sql("SELECT * FROM movie_final", conn)

# Se selecciona un usuario para realizar las recomendaciones 
usuarios = pd.read_sql('SELECT distinct (userId) as user_id from ratings_final', conn)

# Seleccionamos un usuario para realizar el sistema de recomendación
user_id = 605
movies
ratings = pd.read_sql('SELECT * FROM ratings_final', conn)

In [5]:
usuarios

,user_id
0,1
1,4
2,6
3,7
4,10
...,...
369,605
370,606
371,607
372,608


In [6]:

def recomendar(user_id=list(usuarios['user_id'].value_counts().index)):
    
    # Se seleccionan los ratings del usuario establecido
    ratings = pd.read_sql('SELECT * from ratings_final where userId=:user', conn, params={'user': user_id})
    
    #Seleccionar solo peliculas con rating mayor a 4
    ratings = ratings[ratings['rating'] > 4]
    
    # Convertir ratings del usuario a array
    ratings_array = ratings['movieId'].to_numpy()
    
    # Agregar la columna de movieId y título al dataframe df_dum para filtrar y mostrar nombres
    df_dum[['movieId', 'title']] = movies[['movieId', 'title']]
    
    # Filtrar películas calificadas por el usuario
    movies_r = df_dum[df_dum['movieId'].isin(ratings_array)]
    
    # Eliminar columnas movieId y title
    movies_r = movies_r.drop(columns=['movieId', 'title'])
    movies_r["indice"] = 1  # Para usar group by y que quede en formato pandas tabla de centroide
    
    # Centroide o perfil del usuario
    centroide = movies_r.groupby("indice").mean()
    
    # Filtrar películas no leídas
    movies_ns = df_dum[~df_dum['movieId'].isin(ratings_array)]
    
    # Eliminar columnas movieId y title
    movies_ns = movies_ns.drop(columns=['movieId', 'title'])
    
    # Entrenar modelo 
    model = neighbors.NearestNeighbors(n_neighbors=10, metric='cosine')
    model.fit(movies_ns)
    dist, idlist = model.kneighbors(centroide)
    
    ids = idlist[0]  # Queda en un array anidado, para sacarlo
    recomend_m = movies.loc[ids][['title']]
    leidos = movies[movies['movieId'].isin(ratings_array)][['title', 'movieId']]
    
    return recomend_m

In [7]:
# Llamar a la función con el usuario seleccionado
recomendar(608)


,title
23,Braveheart (1995)
251,"Mummy, The (1999)"
306,"Crouching Tiger, Hidden Dragon (Wo hu cang lon..."
368,Harry Potter and the Prisoner of Azkaban (2004)
184,Jerry Maguire (1996)
53,"Shawshank Redemption, The (1994)"
45,Legends of the Fall (1994)
99,"Silence of the Lambs, The (1991)"
140,Star Wars: Episode V - The Empire Strikes Back...
372,"Bourne Supremacy, The (2004)"


### Top 5 de las películas con el género más visto por el usuario

In [8]:
import pandas as pd

def recomendar_peliculas_por_genero(user_id, conn, top_n=10):
    # Se seleccionan los ratings del usuario establecido
    ratings = pd.read_sql('SELECT * from ratings_final where userId=:user', conn, params={'user': user_id})

    # Obtener el array de movieIds vistos por el usuario
    ratings_array = ratings['movieId'].to_numpy()

    # Filtrar películas vistas por el usuario
    peliculas_vistas = movies[movies['movieId'].isin(ratings_array)]

    # Expandir los géneros de las películas vistas por el usuario (suponiendo que los géneros están separados por "|")
    peliculas_vistas['genres'] = peliculas_vistas['genres'].str.split('|')
    
    # Descomponer géneros en filas individuales (cada fila será una película-genre)
    peliculas_con_generos = peliculas_vistas.explode('genres')

    # Contar cuántas películas ha visto el usuario por género
    generos_contados = peliculas_con_generos['genres'].value_counts()

    # Obtener el género más visto por el usuario
    genero_mas_visto = generos_contados.idxmax()

    # Filtrar películas que el usuario aún no haya visto del género más visto
    peliculas_no_vistas = movies[~movies['movieId'].isin(ratings_array)]
    peliculas_recomendadas = peliculas_no_vistas[peliculas_no_vistas['genres'].str.contains(genero_mas_visto, regex=False)]

    # Seleccionar las primeras `top_n` películas del género más visto
    top_recomendadas = peliculas_recomendadas[['title']].head(top_n)

    return top_recomendadas, genero_mas_visto


In [9]:
# Llamar a la función de recomendación
recomendadas, genero = recomendar_peliculas_por_genero(user_id, conn)

# Mostrar las recomendaciones
print(f"Recomendaciones del género más visto '{genero}':")
print(recomendadas)

Recomendaciones del género más visto 'Adventure':
                                     title
5                         GoldenEye (1995)
21                     Broken Arrow (1996)
29                           Casper (1995)
30                            Congo (1995)
38                       Waterworld (1995)
41  Dumb & Dumber (Dumb and Dumber) (1994)
51                         Stargate (1994)
54           Star Trek: Generations (1994)
63                   Lion King, The (1994)
65                         Maverick (1994)


C:\Users\leeon\AppData\Local\Temp\ipykernel_16136\3725160496.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  peliculas_vistas['genres'] = peliculas_vistas['genres'].str.split('|')


In [20]:
# Limpiar la columna 'genres': eliminar espacios extra y caracteres no deseados
movies['genres'] = movies['genres'].str.strip()

# Crear un widget Output para controlar las salidas
output = Output()

In [21]:
# Función para recomendar películas por género seleccionado y mostrar el rating promedio
def recomendar_peliculas_por_genero(genero_seleccionado, user_id, conn, top_n=10):
    # Se seleccionan los ratings del usuario establecido
    ratings_user = pd.read_sql('SELECT * from ratings_final where userId=:user', conn, params={'user': user_id})

    # Obtener el array de movieIds vistos por el usuario
    ratings_array = ratings_user['movieId'].to_numpy()

    # Filtrar películas vistas por el usuario
    peliculas_vistas = movies[movies['movieId'].isin(ratings_array)]

    # Expandir los géneros de las películas vistas por el usuario
    peliculas_vistas.loc[:, 'genres'] = peliculas_vistas['genres'].str.split('|')
    
    # Descomponer géneros en filas individuales (cada fila será una película-genre)
    peliculas_con_generos = peliculas_vistas.explode('genres')

    # Filtrar películas no vistas del género seleccionado
    peliculas_no_vistas = movies[~movies['movieId'].isin(ratings_array)]
    
    # Convertir el género seleccionado a string y eliminar espacios extra
    genero_seleccionado_str = str(genero_seleccionado).strip()

    # Filtrar las películas recomendadas por el género seleccionado
    peliculas_recomendadas = peliculas_no_vistas[peliculas_no_vistas['genres'].str.contains(genero_seleccionado_str, regex=False)]

    # Calcular el rating promedio para cada película
    rating_promedio = ratings.groupby('movieId')['rating'].mean().reset_index()

    # Unir la tabla de películas recomendadas con su rating promedio
    peliculas_recomendadas = peliculas_recomendadas.merge(rating_promedio, on='movieId', how='left')

    # Seleccionar las primeras top_n películas recomendadas y redondear el rating a 2 decimales
    peliculas_recomendadas['rating'] = peliculas_recomendadas['rating'].round(2)

    # Seleccionar las columnas 'title' y 'rating' para mostrar
    top_recomendadas = peliculas_recomendadas[['title', 'rating']].head(top_n)

    return top_recomendadas

# Obtener todos los géneros únicos de la columna 'genres'
todos_los_generos = movies['genres'].str.split('|').explode().unique()

# Crear la función interactiva con salida controlada
@interact
def mostrar_recomendaciones(genero=todos_los_generos):
    output.clear_output()  # Limpiar la salida anterior
    recomendaciones = recomendar_peliculas_por_genero(genero, user_id, conn)
    with output:
        print(f"Recomendaciones para el género '{genero}':")
        print(recomendaciones)

# Mostrar el widget de salida
display(output)


interactive(children=(Dropdown(description='genero', options=('Adventure', 'Animation', 'Children', 'Comedy', …

Output()

## 4 Sistema de recomendación filtro colaborativo

In [12]:
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
cur.fetchall()

[('ratings',),
 ('movies',),
 ('usuarios_sel',),
 ('movies_sel',),
 ('ratings_final',),
 ('movie_final',),
 ('full_ratings',),
 ('reco',)]

In [13]:
# Obtener datos; se filtran los mayores a 0
rat = pd.read_sql('SELECT * FROM ratings_final WHERE rating > 0', conn)

# Definir escala
reader = Reader(rating_scale=(1, 5))  # Corregí el nombre de la variable 'Reader' por 'reader'

# Leer datos con surprise
data = Dataset.load_from_df(rat[['userId', 'movieId', 'rating']], reader)

# Modelos 
models = [KNNBasic(), KNNWithMeans(), KNNWithZScore(), KNNBaseline()] 
results = {}

# Probar modelos
for model in models:
    CV_scores = cross_validate(model, data, measures=["MAE", "RMSE"], cv=5, n_jobs=-1)
    
    result = pd.DataFrame.from_dict(CV_scores).mean(axis=0).\
             rename({'test_mae':'MAE', 'test_rmse': 'RMSE'})
    results[str(model).split("algorithms.")[1].split("object ")[0]] = result

performance_df = pd.DataFrame.from_dict(results).T
performance_df.sort_values(by='RMSE')

# Se seleccionó el KNNBasic porque tiene mejores métricas en "MAE Y RMSE"

# Definir la grilla de parámetros para la búsqueda de hiperparámetros
param_grid = {
    'sim_options': {
        'name': ['msd', 'cosine'],
        'min_support': [5],
        'user_based': [False, True]
    }
}

gridsearchKNNBasic = GridSearchCV(KNNBasic, param_grid, measures=['rmse'], cv=2, n_jobs=-1)

gridsearchKNNBasic.fit(data)

gridsearchKNNBasic.best_params["rmse"]
gridsearchKNNBasic.best_score["rmse"]
gs_model = gridsearchKNNBasic.best_estimator['rmse']

# Entrenar con todos los datos, y realizar predicciones con el modelo afinado
trainset = data.build_full_trainset()
model1 = gs_model.fit(trainset)
predset = trainset.build_anti_testset()
len(predset)
predictions = gs_model.test(predset)

# Crear base; películas no vistas por usuario + calificación predicha 
predictions_df = pd.DataFrame(predictions) 
predictions_df.shape
predictions_df.head()
predictions_df['r_ui'].unique()
predictions_df.sort_values(by='est', ascending=False)


# Función para recomendar 11 películas a un usuario específico
def recomendaciones(user_id, n_recomend=11):
    predictions_userID = predictions_df[predictions_df['uid'] == user_id].\
                    sort_values(by="est", ascending = False).head(n_recomend)

    recomendados = predictions_userID[['iid', 'est']]
    recomendados.to_sql('reco', conn, if_exists="replace")
    
    # Cambié 'movies_1' por 'movies_sel' según tus tablas
    mov = pd.read_sql('''SELECT b.title FROM reco a LEFT JOIN movie_final b ON a.iid=b.movieId''', conn)
    return mov


Computing the msd similarity matrix...
Done computing similarity matrix.


In [14]:
# Obtener y mostrar las 11 películas recomendadas al usuario 50
peliculas = recomendaciones(user_id=605, n_recomend=10)
print(peliculas)

                                               title
0             Monty Python and the Holy Grail (1975)
1                    Star Trek: First Contact (1996)
2                             Children of Men (2006)
3  Star Wars: Episode V - The Empire Strikes Back...
4                               28 Days Later (2002)
5   Pan's Labyrinth (Laberinto del fauno, El) (2006)
6                                          Up (2009)
7                               Prestige, The (2006)
8                   Shawshank Redemption, The (1994)
9                         Million Dollar Baby (2004)
